In [179]:
from datetime import datetime

import pandas as pd
import requests
import time

from api_key import api_key

### Stops

In [5]:
url = "https://api.um.warszawa.pl/api/action/dbstore_get"
params = {
    "id": "ab75c33d-3a26-4342-b36a-6e5fef0a3ac3",  # resource id given by the API
    "apikey": api_key,
}
stops = requests.get(url, params=params)
print(len(stops.json()["result"]))

8241


### Routes

In [6]:
url = "https://api.um.warszawa.pl//api/action/public_transport_routes"
params = {
    "apikey": api_key,
}
routes = requests.get(url, params=params)
print(len(routes.json()["result"]))

287


### Location

In [25]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}
locations = requests.get(url, params=params)
print(len(locations.json()["result"]))

1893


### Timetable

In [8]:
url = "https://api.um.warszawa.pl//api/action/dbtimetable_get"
params = {
    "id": "e923fa0e-d96c-43f9-ae6e-60518c9f3238",  # resource id given by the API
    "apikey": api_key,
    "busstopId": "7074",
    "line": "159",
    "busstopNr": "02",
}
timetable = requests.get(url, params=params)
print(len(timetable.json()["result"]))

51


#### Check what lines of buses and trams are available in api

In [13]:
lines = [i["Lines"] for i in locations.json()["result"]]
lines = list(set(lines))
lines.sort()
print(lines)

['100', '102', '103', '104', '105', '106', '107', '108', '110', '111', '112', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '138', '139', '140', '141', '142', '143', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '197', '198', '199', '201', '202', '203', '204', '207', '208', '209', '210', '211', '212', '213', '217', '218', '219', '220', '221', '226', '228', '234', '239', '240', '245', '250', '251', '256', '262', '303', '308', '311', '317', '331', '338', '340', '401', '402', '409', '411', '414', '500', '502', '503', '504', '507', '509', '511', '512', '514', '516', '517', '518', '519', '520'

In [15]:
lines = [i["Lines"] for i in locations.json()["result"]]
lines = list(set(lines))
lines.sort()
print(lines)

['1', '10', '11', '13', '14', '15', '16', '17', '18', '2', '20', '22', '23', '24', '25', '26', '27', '28', '3', '31', '33', '36', '4', '6', '7', '79', '9']


In [80]:
stops_df = [
    {item["key"]: item["value"] for item in entry["values"]}
    for entry in stops.json()["result"]
]

# Convert to DataFrame
stops_df = pd.DataFrame(stops_df)

In [178]:
routes_df = []
for line_number, line_variants in routes.json()["result"].items():
    for line_variant, line_variant_details in line_variants.items():
        for stop_numer, stop_details in line_variant_details.items():
            stop_details["line_number"] = line_number
            stop_details["line_variant"] = line_variant
            stop_details["stop_numer"] = stop_numer
            routes_df.append(stop_details)

routes_df = pd.DataFrame(routes_df)

set([i.split("-")[0] for i in list(routes_df["line_variant"])])

{'TD', 'TO', 'TP', 'TX', 'TZ'}

In [162]:
locations_df = pd.json_normalize(locations.json()["result"])

In [112]:
timetable_df = [
    {item["key"]: item["value"] for item in entry}
    for entry in timetable.json()["result"]
]

# Convert to DataFrame
timetable_df = pd.DataFrame(timetable_df)

### NEEDS TO INSTALL fastparquet 

In [163]:
locations_df.to_parquet(
    "locations_test.parquet",
    compression="brotli",
)

In [181]:
last_response = requests.get(url, params=params).json()
initial_time = datetime.now()
for i in range(20):
    time.sleep(8)
    response = requests.get(url, params=params).json()
    if response != last_response:
        print(f"{datetime.now()} New response after {datetime.now() - initial_time}")
        last_response = response
        initial_time = datetime.now()
    else:
        print(f"{datetime.now()} Same response after {datetime.now() - initial_time}")

2024-12-29 14:44:00.263255 Same response after 0:00:08.277933
2024-12-29 14:44:08.547868 New response after 0:00:16.561155
2024-12-29 14:44:17.044874 New response after 0:00:08.488571
2024-12-29 14:44:25.309830 New response after 0:00:08.264481
2024-12-29 14:44:35.700726 New response after 0:00:10.388834
2024-12-29 14:44:43.875483 Same response after 0:00:08.170339
2024-12-29 14:44:52.200596 New response after 0:00:16.495461
2024-12-29 14:45:00.478909 New response after 0:00:08.277732
2024-12-29 14:45:08.787390 New response after 0:00:08.307929
2024-12-29 14:45:17.064930 Same response after 0:00:08.276136
2024-12-29 14:45:25.316945 New response after 0:00:16.528163
2024-12-29 14:45:36.373961 New response after 0:00:11.055254
2024-12-29 14:45:44.657046 New response after 0:00:08.281249
2024-12-29 14:45:53.151830 New response after 0:00:08.493516
2024-12-29 14:46:01.420107 Same response after 0:00:08.266077
2024-12-29 14:46:09.679052 New response after 0:00:16.525018
2024-12-29 14:46:17.

In [186]:
last_response = requests.get(url, params=params).json()
initial_time = datetime.now()
for i in range(30):
    time.sleep(1)
    response = requests.get(url, params=params).json()
    if response != last_response:
        print(f"{datetime.now()} New response after {datetime.now() - initial_time}")
        last_response = response
        initial_time = datetime.now()
    else:
        print(f"{datetime.now()} Same response after {datetime.now() - initial_time}")

2024-12-29 14:48:35.698375 Same response after 0:00:03.643327
2024-12-29 14:48:37.012618 Same response after 0:00:04.957493
2024-12-29 14:48:38.732841 Same response after 0:00:06.677721
2024-12-29 14:48:39.989910 New response after 0:00:07.934788
2024-12-29 14:48:41.475202 Same response after 0:00:01.484785
2024-12-29 14:48:42.784930 Same response after 0:00:02.794499
2024-12-29 14:48:44.175853 Same response after 0:00:04.185455
2024-12-29 14:48:45.542775 Same response after 0:00:05.552338
2024-12-29 14:48:46.838299 Same response after 0:00:06.847865
2024-12-29 14:48:48.094639 Same response after 0:00:08.104204
2024-12-29 14:48:49.342866 Same response after 0:00:09.352431
2024-12-29 14:48:50.683170 New response after 0:00:10.692735
2024-12-29 14:48:51.950188 Same response after 0:00:01.266465
2024-12-29 14:48:53.220051 Same response after 0:00:02.536321
2024-12-29 14:48:54.485290 Same response after 0:00:03.801560
2024-12-29 14:48:55.783030 Same response after 0:00:05.099309
2024-12-29